In [ ]:

# 1. pytest 
# 2. cache
# 3. pydantic object and output parser
# 4. token counter(cost analysis)
# 5. history or memory 
# 6. evaluation
# 7. guadrails
# 8. adavance RAG

## as a experiment will do in the notebook
### then as a assignment you can integrate it in end to end project

## Next Saturday there will demonstration of this concept (whoever will comeplete it
# that person will give the demo)

### next week satuday 2-2:30

### then will start with the next project

#For first winner there is prize money:

Assignment on first Project:
1. you have to make enable this project for every document(.ppt,.docx,.md,.txt,.pdf,.xlxs,.csv,anysqldb)
2. you have to add a code for dealing with table and images data also
3. you have to add the evalation matrix using the DeepEval
4. write at least 10 test cases
5. this cases should be vaildate before the commit and after the commit
6. add one login screen to your protal
7. use the langchain inmemory cache and implement inside the project
8. deadline for this assignment is till 5th of september(friday)
9. inner will present the entire solution to the whole class (30 minute presentation would be there)
10. prize: money or course or giftpack(there will be only one winnner)

In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from utils.model_loader import ModelLoader

In [3]:
loader = ModelLoader()

{"timestamp": "2025-08-24T07:43:44.298823Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-08-24T07:43:44.299812Z", "level": "info", "event": "Loaded API_KEYS from ECS secret"}
{"keys": {"GROQ_API_KEY": "gsk_m0...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-08-24T07:43:44.300812Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["faiss_db", "embedding_model", "retriever", "llm"], "timestamp": "2025-08-24T07:43:44.304824Z", "level": "info", "event": "YAML config loaded"}


In [5]:
llm = loader.load_llm()
print(f"LLM Loaded: {llm}")
result = llm.invoke("Hello, how are you?")
print(f"LLM Result: {result.content}")


{"provider": "google", "model": "gemini-2.0-flash", "timestamp": "2025-08-24T07:44:58.301931Z", "level": "info", "event": "Loading LLM"}


LLM Loaded: model='models/gemini-2.0-flash' google_api_key=SecretStr('**********') temperature=0.0 max_output_tokens=2048 client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000018A7A81B070> default_metadata=() model_kwargs={}
LLM Result: I am doing well, thank you for asking! As a large language model, I don't experience emotions or feelings like humans do, but I am functioning optimally and ready to assist you. How can I help you today?


In [6]:
parser = JsonOutputParser()

In [12]:
parser.get_format_instructions()

'Return a JSON object.'

In [7]:
prompt = PromptTemplate.from_template(
    "Give me a JSON with keys 'title' and 'summary' for this topic: {topic}\n{format_instructions}"
).partial(format_instructions=parser.get_format_instructions())


In [8]:
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template="Give me a JSON with keys 'title' and 'summary' for this topic: {topic}\n{format_instructions}")

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template="Give me a JSON with keys 'title' and 'summary' for this topic: {topic}\n{format_instructions}")

In [9]:
chain = prompt | llm | parser

In [10]:
result = chain.invoke({"topic": "LangChain for RAG"})

In [11]:
result

{'title': 'LangChain for Retrieval Augmented Generation (RAG)',
 'summary': 'LangChain simplifies the development of Retrieval Augmented Generation (RAG) applications. It provides modules for connecting to various data sources, creating embeddings, implementing retrieval strategies, and integrating with large language models (LLMs) to generate contextually relevant and informed responses. LangChain streamlines the RAG pipeline, enabling developers to build powerful question answering, chatbot, and knowledge-based systems more efficiently.'}

In [13]:
from langchain.output_parsers import OutputFixingParser

In [14]:
json_parser = JsonOutputParser()

In [15]:
# Fixing wrapper
fixing_parser = OutputFixingParser.from_llm(parser=json_parser, llm=llm)

In [16]:
# Broken JSON
bad_output = """
title: LangChain RAG
summary LangChain helps with retrieval augmented generation...
"""

In [17]:

fixed_result = fixing_parser.parse(bad_output)

In [18]:
print(fixed_result)

{'title': 'LangChain RAG', 'summary': 'LangChain helps with retrieval augmented generation...'}


| Feature                | `JsonOutputParser`               | `OutputFixingParser`                     
| ---------------------- | -------------------------------- | ---------------------------------------- 
| Purpose                | Enforce strict JSON              | Auto-correct bad/invalid output          
| Works with             | Well-formed model responses only | Even with malformed model responses      
| Raises error if broken | Yes                            |   No, it tries to fix using LLM          
| Ideal for              | Strict structured output         | Fragile prompts, semi-structured outputs 



In [ ]:
# from langchain.output_parsers import OutputFixingParser
# from langchain_core.output_parsers import JsonOutputParser

# json_parser = JsonOutputParser()
# safe_parser = OutputFixingParser.from_llm(parser=json_parser, llm=llm)

# # Use safe_parser instead of json_parser
# chain = prompt | llm | safe_parser


In [22]:
# Wrap with cache
Model_Cache = {}

import time
def cached_model(query):
    start_time = time.time()
    if Model_Cache.get(query):
        print("***CACHE HIT***")
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        return Model_Cache.get(query)
    else:
        print("***CACHE MISS – EXECUTING MODEL***")
        start_time = time.time()
        response = llm.invoke(query)
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        Model_Cache[query] = response
        return response

In [24]:
query="hi"
response = cached_model(query)
print(response)

***CACHE HIT***
EXECUTION TIME: 0.00 seconds
content='Hi there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--2b56fd6b-b818-4b2c-a48a-8295d36db733-0' usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}}


In [25]:
query="can you give me 1000 words essay on independence?"
response = cached_model(query)
print(response)

***CACHE MISS – EXECUTING MODEL***
EXECUTION TIME: 8.92 seconds
content="## The Enduring Quest for Independence: A Multifaceted Human Imperative\n\nIndependence, a word that resonates with the spirit of freedom and self-determination, is a fundamental human aspiration. It is a concept that permeates our lives, shaping our individual identities, influencing our societal structures, and driving historical movements. From the toddler learning to walk to the nation striving for sovereignty, the pursuit of independence is a constant and multifaceted journey, encompassing personal autonomy, economic self-sufficiency, and political liberation. This essay will explore the various dimensions of independence, examining its significance, its challenges, and its enduring relevance in the modern world.\n\nAt its core, independence is about the ability to make choices and act upon them without undue external control. This begins at the individual level, where the development of personal autonomy is 

In [26]:
query="can you give me 1000 words essay on independence?"
response = cached_model(query)
print(response)

***CACHE HIT***
EXECUTION TIME: 0.00 seconds
content="## The Enduring Quest for Independence: A Multifaceted Human Imperative\n\nIndependence, a word that resonates with the spirit of freedom and self-determination, is a fundamental human aspiration. It is a concept that permeates our lives, shaping our individual identities, influencing our societal structures, and driving historical movements. From the toddler learning to walk to the nation striving for sovereignty, the pursuit of independence is a constant and multifaceted journey, encompassing personal autonomy, economic self-sufficiency, and political liberation. This essay will explore the various dimensions of independence, examining its significance, its challenges, and its enduring relevance in the modern world.\n\nAt its core, independence is about the ability to make choices and act upon them without undue external control. This begins at the individual level, where the development of personal autonomy is crucial for a fulfi

In [27]:
embedding_model = loader.load_embeddings()
print(f"Embedding Model Loaded: {embedding_model}")
result = embedding_model.embed_query("Hello, how are you?")
print(f"Embedding Result: {result}")


{"model": "models/text-embedding-004", "timestamp": "2025-08-24T07:58:48.562915Z", "level": "info", "event": "Loading embedding model"}


Embedding Model Loaded: client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000018A7BAAAAA0> async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x0000018A7BE60BB0> model='models/text-embedding-004' task_type=None google_api_key=SecretStr('**********') credentials=None client_options=None transport=None request_options=None
Embedding Result: [0.012455824762582779, -0.022220758721232414, -0.050555989146232605, -0.04617856815457344, 0.007134224288165569, 0.03131191432476044, 0.03413437679409981, 0.002971651265397668, -0.010995007120072842, 0.04237418621778488, -0.01806010864675045, 0.02493678405880928, 0.1121494323015213, -0.013102629221975803, -0.0148635134100914, -0.024023527279496193, 0.010264375247061253, -0.0037090936675667763, -0.10687677562236786, -0.001329770078882575, 0.03868139907717705, -0.03769991546869278, 0.03614252060651779, 0.005920

In [28]:
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [29]:
from langchain.schema import Document

documents = [
    Document(page_content="The Earth is the third planet from the Sun and the only known planet to support life. It has a diverse climate, ranging from arctic to tropical zones, and supports ecosystems across seven continents and five oceans."),

    Document(page_content="The Industrial Revolution, beginning in the 18th century, drastically transformed human societies by shifting from manual labor to machine-based manufacturing, leading to urbanization and economic expansion globally."),

    Document(page_content="The United Nations, established in 1945 after World War II, is an international organization founded to promote peace, security, human rights, and cooperation among countries. It has 193 member states."),

    Document(page_content="The global economy is an interconnected system involving trade, investment, and financial flows across countries. Major players include the United States, China, the European Union, and emerging markets like India and Brazil."),

    Document(page_content="Climate change refers to long-term shifts in temperatures and weather patterns. It is largely driven by human activities like burning fossil fuels, deforestation, and industrial emissions, leading to global warming and sea level rise."),

    Document(page_content="Democracy is a political system in which citizens exercise power by voting. Modern democracies typically have institutions for free elections, rule of law, freedom of expression, and checks and balances."),

    Document(page_content="The Internet has revolutionized communication, commerce, and education worldwide. Originating from military research in the 1960s, it now connects over 5 billion people, enabling instant global information exchange."),

    Document(page_content="Renewable energy sources like solar, wind, hydro, and geothermal are critical for a sustainable future. They offer alternatives to fossil fuels, reducing carbon emissions and reliance on finite resources."),

    Document(page_content="The World Health Organization (WHO) is a UN agency focused on global health issues. It coordinates international efforts to monitor diseases, set health standards, and respond to pandemics like COVID-19."),

    Document(page_content="Globalization is the process of increasing interaction and integration among people, companies, and governments worldwide. It has led to greater economic growth but also raised concerns about inequality and cultural homogenization.")
]

In [30]:
# Chroma vector DB with persistent storage
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # Disk path for persistence
)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
# Optional: Persist manually (though auto-persistence happens internally)
vector_store.persist()

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
prompt = PromptTemplate.from_template(
    """
    Use the following context to answer the question.
    If you don't know the answer, just say you don't know. Don't try to make up an answer.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
)

In [ ]:
# LCEL RAG Chain step-by-step
rag_chain = (
    RunnableMap({
        "context": retriever | (lambda docs: "\n\n".join([doc.page_content for doc in docs])),
        "question": RunnablePassthrough()
    })
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Wrap with cache
RAG_Cache = {}

In [ ]:
import time
def cached_rag_chain(query):
    start_time = time.time()
    if RAG_Cache.get(query):
        print("***CACHE HIT***")
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        return RAG_Cache.get(query)
    else:
        print("***CACHE MISS – EXECUTING MODEL***")
        start_time = time.time()
        response = llm.invoke(query)
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        RAG_Cache[query] = response
        return response

In [ ]:
query = "what is japan economy in 2024 and relation with north korea?"
response = cached_rag_chain(query)
print(response)

In [ ]:
query = "what is langchain framework?"
response = cached_rag_chain(query)
print(response)

In [ ]:
query = "Why United Nations, established in 1945 after World War II?"
response = cached_rag_chain(query)
print(response)

## Cache Using LangChain

In [ ]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from typing import Any, Dict, Tuple

In [ ]:
class DebuggableCache(InMemoryCache):
    def __init__(self):
        super().__init__()
        self._cache: Dict[Tuple[str, str], Any] = {}

    def lookup(self, prompt: str, llm_string: str):
        return self._cache.get((prompt, llm_string))

    def update(self, prompt: str, llm_string: str, return_val: Any):
        self._cache[(prompt, llm_string)] = return_val

    def view_cache(self):  # this is our custom method
        return self._cache

In [ ]:
dbg_cache = DebuggableCache()
set_llm_cache(dbg_cache)

In [ ]:
response = llm.invoke("What is the capital of France?")

In [ ]:
print("LLM Response:", response)

In [ ]:
print("\nCache Contents:")
for k, v in dbg_cache.view_cache().items():
    print(f"Prompt: {k[0]} | Cached Output: {v}")

In [ ]:
response = llm.invoke("What is the capital of France?")

In [ ]:
print("\nCache Contents:")
for k, v in dbg_cache.view_cache().items():
    print(f"Prompt: {k[0]} | Cached Output: {v}")

In [ ]:
response = llm.invoke("What is the capital of india?")
print("LLM Response:", response)

In [ ]:
print("\nCache Contents:")
for k, v in dbg_cache.view_cache().items():
    print(f"Prompt: {k[0]} | Cached Output: {v}")

In [ ]:
response = llm.invoke("What is the capital of india?")
print("LLM Response:", response)

In [ ]:
response = llm.invoke("give me 1000 lines of essay on science and give the importance of it regarding mathematics?")
print("LLM Response:", response)

In [33]:
from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    response = llm.invoke("Tell me a joke about LangChain and ECS")
    print("Response:", response.content)
    print("Token Usage Stats:", cb)

Response: Okay, here's a joke about LangChain and ECS:

Why did the LangChain agent refuse to deploy on ECS?

Because it kept saying, "I can't find the context! I need more environment variables! And where's my vector database? This feels like a container without a purpose... I'm going back to my Jupyter notebook!"
Token Usage Stats: Tokens Used: 84
	Prompt Tokens: 9
		Prompt Tokens Cached: 0
	Completion Tokens: 75
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0


In [34]:
questions = [
    "What is Retrieval-Augmented Generation?",
    "How does FAISS indexing work?",
    "What is the difference between fine-tuning and RAG?"
]

with get_openai_callback() as cb:
    for q in questions:
        answer = llm.invoke(q)
        print(f"\nQ: {q}\nA: {answer.content}")

    print("\n=== Token Usage Summary ===")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost:.6f}")



Q: What is Retrieval-Augmented Generation?
A: Retrieval-Augmented Generation (RAG) is a technique that enhances the capabilities of large language models (LLMs) by allowing them to access and incorporate information from external knowledge sources during the generation process.  Think of it as giving an LLM a textbook or a library to consult before answering a question.

Here's a breakdown of the key components and how it works:

**1. The Problem RAG Solves:**

* **LLMs have limited knowledge:** LLMs are trained on massive datasets, but their knowledge is frozen in time. They can't access real-time information or specific domain knowledge that wasn't included in their training data.
* **Hallucinations:** LLMs can sometimes generate incorrect or nonsensical information, often referred to as "hallucinations." This happens because they're trying to answer questions based on patterns learned during training, even if they don't have the actual facts.
* **Lack of Transparency:** It's often 